<a href="https://colab.research.google.com/github/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/WilliamSteveRodriguezVillamizar_Seminario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Elección de grupos de población homogéneos <br>

Descripción del problema:(copiar enunciado)

Elección de grupos de población homogéneos
• Una productora ganadera nos encarga la tarea de seleccionar grupos de
terneros para aplicar 3 tratamientos diferentes. Para cada uno de los
tratamientos debemos seleccionar 3 grupos de terneros separados por sexos
que sean lo mas homogéneos posible en peso para que en los resultados del
tratamiento influya lo menos posible el peso del animal. Disponemos de una
población de N animales entre machos y hembras
• Se solicita diseñar un algoritmo para conseguir una agrupación que cumpla de la
mejor manera posible las especificaciones de la productora.

....

(*) La respuesta es obligatoria





                                        

In [1]:
!pip install sorting

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [0]:
#Sin ninguna restricción hay 2^n posibles soluciones, donde n es el numero de terneros en el dataset, 
#pero teniendo las restricciones y mirando que el tamaño de datos es limitado en el dataset, 
#y tambien mirando que el sistema solicita grupos homogeneos se generan tambien 2^x posibilidades, 
#donde x es la menor cantidad de inviduos ya sea machos o hembras, siempre y cuando los datos del dataset tengan pesos similares

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

In [0]:
#La estructura mas óptima sea aquella donde los pesos individuales de cada ternero sea muy cercano a la media, 
#puesto que asi, todos serian aptos para usarse en los grupos homogeneos sin descartar a terneros con mucho peso o poco peso, por la restricción que solicita el problema

#Para este ejercicio, dado que no se contaba con un dataset inicial, se ópto por generar un dataset aleatorio lo mas real posible
#para ello se indago sobre los pesos de terneros en el mercado, encontrandose que los terneros que se usan para este tipo de pruebas varian desde los 8 a los 12 meses
#asi que se investigo los pesos para estas edades y se parametrizo esto en un diccionario (edadRangoPeso)
#aleatoriamente genero un sexo para el ternero
#seguidamente el algoritmo que crea este dataset, elije una de estas edades aleatoriamente y con esto, nuevamente aleatorio, genero un peso al azar, 
#pero como bien es sabido que en la naturaleza el macho es mas pesado que la hembra (en la mayoria de las veces),
#si el sexo del ternero es macho, aleatoriamente elijo un porcentaje entre 3% y 10%, para incrementar el peso generado aleatoriamente
#esto para hacer mas cercano a la realidad este dataset
#finalmente para cada item del dataset (se generar tantos terneros como lo defina el usuario) se genera una etiqueta (diferente al index de la tabla),
#simulando asi el marcado que se le hace a los terneros en las granjas, generalmente en la oreja, para luego identificarlo
#y en este caso, decir que el ternero etiquetado_con_con_el_codigo_XXX ira a un grupo u otro de tal tratamiento.

#Esto es una manera algo practica, donde se crean terneros al azar, pero sin salirse de la realidad de los terneros.
#sin embargo, como es un metodo donde se usa la recursividad y el azar, se generan terneros muy gordos y otros muy flacos, 
#con lo cual estos mayormente no se usaran en la selección por alterar la homogeneidad de los grupos

#Pero aún asi, una manera mas eficiente de generar este dataset sería usar un algoritmo genetico donde con un padre, generar hijos (con los cuales llenariamos el dataset) 
#lo mas parecidos al genoma del padre, con esto podriamos aprovechar mas cantidad de muestras del dataset, y asi el algoritmo daria resultados mas interesantes 
#a nivel de cantidad de sujetos para cada tratamiento

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

In [0]:
#Este problema de minización es un ejemplo claro de efectuar ordenamiento sort y busqueda heuristica para encontrar a los mejores elementos a utilizar en la prueba 
#en otras palabras, los terneros con los mejores genes 
#(escribiendo esto veo que un algoritmo genetico hubiera sido interesante para generar el dataset, pero solo se me ocurrio cuando me pregunte: Que otras formas hay de crear el dataset... sera para la proxima)

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

In [0]:
#O(n log n)

ok - (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [0]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from time import time

tiempoInicial = time()

In [0]:
#Decorador de Ricardo Lebron
from functools import wraps
from time import time

def calcular_tiempo(f):
    @wraps(f)
    def cronometro(*args, **kwargs):
        t_inicial = time() #tomo la hora antes de ejecutar la funcion
        salida = f(*args, **kwargs)
        t_final = time() #tomo la hora despues de ejecutar la funcion
        print('Tiempo transcurrido (en segundos): {}'.format(t_final - t_inicial))
        return salida
    return cronometro

In [0]:
nombreArchivo = "TernerosProductoraGanadera.csv"

sexo = {
    'Macho' : 'M',
    'Hembra' : 'H'
}

## Aunque estas tablas nos muestran el precio, tambien podemos usarlas para tener una guia de los pesos conforme a la edad del ternero, esto para tener datos mas realistas


<img src="https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/pesoTerneros.jpg?raw=1">

<img src="https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/pesoTerneros2.jpg?raw=1">

In [0]:
edadRangoPeso = {
    "8" : (120, 160),
    "9" : (160, 180),
    "10" : (180, 200),
    "11" : (200, 230),
    "12" : (230, 260)
}

In [0]:
NombreColumnas = [
    'EtiquetaNacimientoTernero',
    'Sexo',
    'Edad (meses)',
    'Peso'
]

Grupos = {
    "Grupo1" : "A",
    "Grupo2" : "B",
    "Grupo3" : "C"
}

Tratamientos = [
    "1",
    "2",
    "3"
]


Tratamientos_grupos = {
    "Tratamiento1_grupo1" : Tratamientos[0] + Grupos["Grupo1"],
    "Tratamiento1_grupo2" : Tratamientos[0] + Grupos["Grupo2"],
    "Tratamiento1_grupo3" : Tratamientos[0] + Grupos["Grupo3"],
    
    "Tratamiento2_grupo1" : Tratamientos[1] + Grupos["Grupo1"],
    "Tratamiento2_grupo2" : Tratamientos[1] + Grupos["Grupo2"],
    "Tratamiento2_grupo3" : Tratamientos[1] + Grupos["Grupo3"],    
    
    "Tratamiento3_grupo1" : Tratamientos[2] + Grupos["Grupo1"],
    "Tratamiento3_grupo2" : Tratamientos[2] + Grupos["Grupo2"],
    "Tratamiento3_grupo3" : Tratamientos[2] + Grupos["Grupo3"]
}

distanciaMinimaMedia = 10
numero_minimo_terneros_cada_Tratamientos_grupos = 3

In [0]:
import random
def GenerarTerneroAleatorio(etiquetaTernero = None):
    """
    Vamos a generar una edad al azar usando la libreria random, esto debido a que el enunciado dice que es un ternero, pero 
    un ternero se condidera ternero cuando esta en las edades entre 8 y 12 meses, luego se considera novillo,
    y para cada edad hay un intervalo de peso estandarizado, 
    por ejemplo para un ternero de 10 meses su peso suele estar entre 200Kgr y 230Kgr,
    en este orden de ideas, para generar datos lo mas realistas posibles, 
    vamos a generar terneros con pesos que cumplan los intervalos por edades, 
    luego podemos quitar la columna de edad y quedarnos con el genero (generado aleatoriamente) y el peso.
    """
    edad = random.choice(list(edadRangoPeso.keys()) )
    sexoTernero = sexo[random.choice(list(sexo.keys()) )]
    pesoTernero = random.randint(edadRangoPeso[edad][0] , edadRangoPeso[edad][1])
    if sexoTernero == sexo['Macho']:
        #Generalmente los machos son mas pesados
        #Por este motivo a los machos le damos de un 3% a un 10% mas de peso (valor elegido al azar)
        porcentajePeroMayorPorSerMacho = random.randint(3 , 10) #Elegimos el porcentaje de peso que tendra de mas a comparacion si fuere una hembra
        pesoTernero = pesoTernero + (pesoTernero*porcentajePeroMayorPorSerMacho)/100
        
    ternero = { 
        NombreColumnas[0] : etiquetaTernero,
        NombreColumnas[1] : [sexoTernero] ,
        NombreColumnas[2] : [int(edad)], 
        NombreColumnas[3] : [pesoTernero] 
          }
    return ternero

def GenerarGrupoTerneros(sizeDataframe, dataframe = None):
    """
    Usando recursividad vamos a generar el dataframe del tamaño solicitado en la variable "sizeDataframe"
    llenando el dataframe con terneros aleatorios
    """
    ternero = GenerarTerneroAleatorio(sizeDataframe)
    try:
        temporal = pd.DataFrame(ternero)
        dataframe = dataframe.append(temporal)
    except:
        dataframe = pd.DataFrame(ternero)        
    
    if 0 < sizeDataframe:        
        GenerarGrupoTerneros(sizeDataframe -1, dataframe)
    else:
        export_csv = dataframe.to_csv (nombreArchivo, index = None, header=True) 
        
@calcular_tiempo
def GenerarDataframeDatos(sizeDataframe):   
    """
    Colocamos esta funcion para poder medir el tiempo que toma el algoritmo para crear el conjunto de datos 
    (para terminos evaluativos en cuanto a optimizacion del codigo)
    """
    total = numero_minimo_terneros_cada_Tratamientos_grupos * 18
    if numero_minimo_terneros_cada_Tratamientos_grupos > sizeDataframe:
      sizeDataframe = numero_minimo_terneros_cada_Tratamientos_grupos
    GenerarGrupoTerneros(sizeDataframe)

In [12]:
N_animales = 800

#generamos un dataset aleatorio guardado en el archivo "nombreArchivo" declarado al inicio del codigo
GenerarDataframeDatos(N_animales)

Tiempo transcurrido (en segundos): 2.1228156089782715


In [0]:
#cargamos el dataset que hemos guardado en el .csv

@calcular_tiempo
def LeerArchivo():
    return pd.read_csv(nombreArchivo) 

In [14]:
purchases = LeerArchivo()

Tiempo transcurrido (en segundos): 0.009331226348876953


In [0]:
tiempoProcesar = time()

In [0]:
#dado que la columna de edades solo se uso para generar aleatoriamente los pesos
#conforme a la edad, no es necesario que de aca en adelante la sigamos conservando
#por lo cual eliminamos esta columna del dataframe cargado para facilitar el analisis de los datos
purchases = purchases.drop([NombreColumnas[2]], axis=1) 
purchases.dropna(inplace=True) #Borro valores nulos, si llegaren a existir

In [17]:
@calcular_tiempo
def SepararPorGenero(df, genero):
    return df[df[NombreColumnas[1]] == sexo[genero]]

data_male = SepararPorGenero(purchases, "Macho" )

data_female = SepararPorGenero(purchases, "Hembra" )

#dado que ya hemos separado el conjunto de datos por sexos, ya no necesitamos esa columna, 
#por lo cual la podemos eliminar para facilitar el analisis de los datos
data_male = data_male.drop([NombreColumnas[1]], axis=1)
data_female = data_female.drop([NombreColumnas[1]], axis=1) 

Tiempo transcurrido (en segundos): 0.004439592361450195
Tiempo transcurrido (en segundos): 0.0030667781829833984


In [18]:
#ordenamos los conjunto de datos para analizar mas facil la información

@calcular_tiempo
def OrdenarPorPeso(df):
    return df.sort_values(by=[ NombreColumnas[3] ])

data_male = OrdenarPorPeso(data_male)
data_female = OrdenarPorPeso(data_female)

Tiempo transcurrido (en segundos): 0.0031485557556152344
Tiempo transcurrido (en segundos): 0.002602100372314453


In [0]:
@calcular_tiempo
def GruposPorTratamiento(df):
    #print("Tamaño",df.shape[0])    
    #df = df.sort_index()    
    split = int(round(df.shape[0]/3)-1)
    grupoTratamiento = {}
    
    """
    Dividimos el dataframe para tres tratamientos creando 3 grupos
    """
    for i in range(len(Tratamientos)):
        tratamientto = Tratamientos[i]
        grupoTratamiento[tratamientto] = df[:split]
        df = df[split:]
    
    """
    Para cada tratamiento, genero tres grupos, tratando en lo mas posible que mantengan la media en los mismos
    """
    grupos = {}
    for keyTratamiento in grupoTratamiento:
        grupo_para_tratamiento =  grupoTratamiento[keyTratamiento]
        grupo_para_tratamiento = grupo_para_tratamiento.sort_index()
        split = int(round(grupo_para_tratamiento.shape[0]/3)-1)
        #print(keyTratamiento, grupo_para_tratamiento.mean()[1])
        
        for keyGrupo in Grupos:
            llave = Grupos[keyGrupo]
            grupos[keyTratamiento+llave] = grupo_para_tratamiento[:split]
            grupo_para_tratamiento = grupo_para_tratamiento[split:]
        
    return grupos   

In [20]:
"""
Ahora tenermos dos subconjunto de datos (machos y hembras)
Vamos a dividir cada subconjunto en 9, asi tendremos 3 grupos para cada uno de los tres tratamientos
Seguidamente vamos a obtener la media de cada uno de estos 9 grupos y vamos a obtener la distancia de cada elemento del grupo a esta media
Haremos lo mismo tanto para los masculinos como para el conjunto de datos femeninos
"""

grupos_data_male = GruposPorTratamiento(data_male)
grupos_data_female = GruposPorTratamiento(data_female)

Tiempo transcurrido (en segundos): 0.011604547500610352
Tiempo transcurrido (en segundos): 0.008509159088134766


In [0]:
@calcular_tiempo
def LimpiarOrganizarGruposDatos(df, mediaMinima):
    """
    Cada uno de los nueve grupos, los organizo por cual dato esta mas cercano a la media
    """
    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        dataframe = df[llave]
        MediaDatos = dataframe.mean()[1] #calcular la media de los datos de este grupo

        dataframe['DistanciaMedia'] = abs(MediaDatos - dataframe[NombreColumnas[3]]) #hallo la distancia a la media
        
        busquedaMinimoUno = True
        incrementoRestriccion = 0
        while busquedaMinimoUno:            
            ternero_que_cumple_con_restriccion = dataframe['DistanciaMedia'] <= (mediaMinima + incrementoRestriccion)
            temporal = dataframe[ternero_que_cumple_con_restriccion] #verifico que tenga la distancia minima a la media
            
            if temporal.shape[0] >(numero_minimo_terneros_cada_Tratamientos_grupos - 1):
                dataframe = temporal
                busquedaMinimoUno = False   
            else:
                incrementoRestriccion += 1
        
        dataframe = dataframe.sort_values(by=[ 'DistanciaMedia' ]) #ordeno segun quien este mas cercano a la media
        dataframe = dataframe.drop(['DistanciaMedia'], axis=1)  #quitamos la columna añadida

        df[llave] = dataframe 
    return df

In [22]:
"""
Para cada grupo de cada tratamiento:
Optenemos la media
comparamos cada peso con esta media y hallamos la distancia encomparacion con la media
ordeno los datos conforme a los que tengan menor distancia a la media
"""

grupos_data_male = LimpiarOrganizarGruposDatos(grupos_data_male, distanciaMinimaMedia)
grupos_data_female = LimpiarOrganizarGruposDatos(grupos_data_female, distanciaMinimaMedia)

Tiempo transcurrido (en segundos): 0.5931625366210938
Tiempo transcurrido (en segundos): 0.5652101039886475


In [23]:
#Vamos a hallar la menor cantidad para cada grupo, esto para que todos los grupos queden del mismo tamaño

@calcular_tiempo
def determinarDimensionCadaGrupo():
    cantidad_grupo_mas_pequeno = purchases.shape[0]
    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        if cantidad_grupo_mas_pequeno > grupos_data_male[llave].shape[0]:
            cantidad_grupo_mas_pequeno = grupos_data_male[llave].shape[0]

    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        if cantidad_grupo_mas_pequeno > grupos_data_female[llave].shape[0]:
            cantidad_grupo_mas_pequeno = grupos_data_female[llave].shape[0]
    
    return cantidad_grupo_mas_pequeno
            
cantidad_grupo_mas_pequeno = determinarDimensionCadaGrupo()

Tiempo transcurrido (en segundos): 0.00011205673217773438


In [24]:
@calcular_tiempo
def RecortarGruposDatos(df, corte):
    """
    Cada uno de los nueve grupos, los organizo por cual dato esta mas cercano a la media
    """
    for key in Tratamientos_grupos:     
        llave = Tratamientos_grupos[key]
        dataframe = df[llave]
        dataframe = dataframe[:corte]
        df[llave] = dataframe
    return df

grupos_data_male = RecortarGruposDatos(grupos_data_male, cantidad_grupo_mas_pequeno)
grupos_data_female = RecortarGruposDatos(grupos_data_female, cantidad_grupo_mas_pequeno)

Tiempo transcurrido (en segundos): 0.0041637420654296875
Tiempo transcurrido (en segundos): 0.0041043758392333984


In [0]:
tiempoFinal = time()

In [26]:
conteo = 0
for key in Tratamientos_grupos:     
    llave = Tratamientos_grupos[key]    
    if conteo >= 3:
      conteo = 0
      print("")
    conteo += 1
    print("Male: ", llave, " terneros: " ,grupos_data_male[llave].shape[0],"Media grupo: " , round(grupos_data_male[llave].mean()[1]) )
    
print()
conteo = 0
for key in Tratamientos_grupos:     
    llave = Tratamientos_grupos[key]
    if conteo >= 3:
      conteo = 0
      print("")
    conteo += 1
    print("Female: ", llave, " terneros: " ,grupos_data_female[llave].shape[0] ,"Media grupo: " , round(grupos_data_female[llave].mean()[1]) )

print()
print("Terneros total utilizados: ", cantidad_grupo_mas_pequeno * 18, ", de un total de ", N_animales)

Male:  1A  terneros:  9 Media grupo:  162.0
Male:  1B  terneros:  9 Media grupo:  160.0
Male:  1C  terneros:  9 Media grupo:  157.0

Male:  2A  terneros:  9 Media grupo:  201.0
Male:  2B  terneros:  9 Media grupo:  197.0
Male:  2C  terneros:  9 Media grupo:  198.0

Male:  3A  terneros:  9 Media grupo:  249.0
Male:  3B  terneros:  9 Media grupo:  246.0
Male:  3C  terneros:  9 Media grupo:  244.0

Female:  1A  terneros:  9 Media grupo:  151.0
Female:  1B  terneros:  9 Media grupo:  151.0
Female:  1C  terneros:  9 Media grupo:  147.0

Female:  2A  terneros:  9 Media grupo:  194.0
Female:  2B  terneros:  9 Media grupo:  196.0
Female:  2C  terneros:  9 Media grupo:  191.0

Female:  3A  terneros:  9 Media grupo:  236.0
Female:  3B  terneros:  9 Media grupo:  241.0
Female:  3C  terneros:  9 Media grupo:  233.0

Terneros total utilizados:  162 , de un total de  800


In [27]:
print('Tiempo transcurrido generar dataset (en segundos): {}'.format(tiempoProcesar - tiempoInicial))
print('Tiempo transcurrido generar respuesta problema (en segundos): {}'.format(tiempoFinal - tiempoProcesar))

Tiempo transcurrido generar dataset (en segundos): 2.2683916091918945
Tiempo transcurrido generar respuesta problema (en segundos): 1.3600542545318604


<img src="https://github.com/wisrovi/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/Seminario/ternero.jpg?raw=1">

(*)Calcula la complejidad del algoritmo 

Respuesta

In [0]:
#O(n^2)

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [29]:
N_animales = 800

#generamos un dataset aleatorio guardado en el archivo "nombreArchivo" declarado al inicio del codigo
GenerarDataframeDatos(N_animales) #esta funcion usada antes me guarda los datos en un archivo

LeerArchivo()

Tiempo transcurrido (en segundos): 1.9210925102233887
Tiempo transcurrido (en segundos): 0.003945350646972656


,EtiquetaNacimientoTernero,Sexo,Edad (meses),Peso
0,800,H,12,244.00
1,799,H,8,123.00
2,798,M,12,281.22
3,797,M,9,181.90
4,796,M,11,226.80
5,795,H,12,240.00
6,794,H,11,200.00
7,793,M,8,164.85
8,792,M,12,278.64
9,791,M,9,188.68


Aplica el algoritmo al juego de datos generado

Respuesta

In [30]:
purchases = LeerArchivo()

purchases = purchases.drop([NombreColumnas[2]], axis=1) 
purchases.dropna(inplace=True)

data_male = SepararPorGenero(purchases, "Macho" )
data_female = SepararPorGenero(purchases, "Hembra" )

data_male = data_male.drop([NombreColumnas[1]], axis=1)
data_female = data_female.drop([NombreColumnas[1]], axis=1)

data_male = OrdenarPorPeso(data_male)
data_female = OrdenarPorPeso(data_female)

grupos_data_male = GruposPorTratamiento(data_male)
grupos_data_female = GruposPorTratamiento(data_female)

grupos_data_male = LimpiarOrganizarGruposDatos(grupos_data_male, distanciaMinimaMedia)
grupos_data_female = LimpiarOrganizarGruposDatos(grupos_data_female, distanciaMinimaMedia)

cantidad_grupo_mas_pequeno = determinarDimensionCadaGrupo()

grupos_data_male = RecortarGruposDatos(grupos_data_male, cantidad_grupo_mas_pequeno)
grupos_data_female = RecortarGruposDatos(grupos_data_female, cantidad_grupo_mas_pequeno)

print()
conteo = 0
for key in Tratamientos_grupos:     
    llave = Tratamientos_grupos[key]    
    if conteo >= 3:
      conteo = 0
      print("")
    conteo += 1
    print("Male: ", llave, " terneros: " ,grupos_data_male[llave].shape[0],"Media grupo: " , round(grupos_data_male[llave].mean()[1]) )
    
print()
conteo = 0
for key in Tratamientos_grupos:     
    llave = Tratamientos_grupos[key]
    if conteo >= 3:
      conteo = 0
      print("")
    conteo += 1
    print("Female: ", llave, " terneros: " ,grupos_data_female[llave].shape[0] ,"Media grupo: " , round(grupos_data_female[llave].mean()[1]) )

print()
print("Terneros total utilizados: ", cantidad_grupo_mas_pequeno * 18, ", de un total de ", N_animales)

Tiempo transcurrido (en segundos): 0.005388021469116211
Tiempo transcurrido (en segundos): 0.001954793930053711
Tiempo transcurrido (en segundos): 0.0010385513305664062
Tiempo transcurrido (en segundos): 0.0006928443908691406
Tiempo transcurrido (en segundos): 0.000644683837890625
Tiempo transcurrido (en segundos): 0.005186796188354492
Tiempo transcurrido (en segundos): 0.005102634429931641
Tiempo transcurrido (en segundos): 0.7711608409881592
Tiempo transcurrido (en segundos): 0.5753850936889648
Tiempo transcurrido (en segundos): 8.96453857421875e-05
Tiempo transcurrido (en segundos): 0.0025081634521484375
Tiempo transcurrido (en segundos): 0.001995563507080078

Male:  1A  terneros:  10 Media grupo:  163.0
Male:  1B  terneros:  10 Media grupo:  162.0
Male:  1C  terneros:  10 Media grupo:  156.0

Male:  2A  terneros:  10 Media grupo:  200.0
Male:  2B  terneros:  10 Media grupo:  201.0
Male:  2C  terneros:  10 Media grupo:  201.0

Male:  3A  terneros:  10 Media grupo:  252.0
Male:  3B  

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta:
Para mejorar el algoritmo se podria utilizar algoritmos geneticos para generar el dataset

Y para su solucion se podria utilizar algoritmos heuristicos que da soluciones mas rapidas y algunas veces (al aplicar un poco soluciones aleatorias) logra soluciones mejores a un algoritmo cerrado como el diseñado en este trabajo.